In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

base_url = "https://www.thriftbooks.com/browse/?13362col#b.s=mostPopular-desc&b.p={}&b.pp=50&b.col&b.f.t%5B%5D=13362&b.list"
output_file = 'thriftBooks.csv'

# write CSV headers once
with open(output_file, 'w') as file:
    file.write('Name,Author,Publisher,LowPrice,HighPrice,ISBN,QualityDescription\n')

for page in range(1, 20):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    scripts = soup.find_all('script')
    script_with_data = None

    # find the script that contains 'window.searchStoreV2'
    for script in scripts:
        if script.string and 'window.searchStoreV2' in script.string:
            script_with_data = script.string
            break

    if not script_with_data:
        print(f"No data found on page {page}")
        continue

    match = re.search(
        r'window\.searchStoreV2\s*=\s*({.*?});', 
        script_with_data, 
        re.DOTALL
    )

    if not match:
        print(f"Regex failed on page {page}")
        continue

    try:
        raw_data = match.group(1)
        data_json = json.loads(raw_data)
        data = data_json.get('works', [])
    except Exception as e:
        print(f"JSON error on page {page}:", e)
        continue

    with open(output_file, 'a', encoding='utf-8') as file:
        for item in data:
            name = item.get('title', '').replace(',', ' ')
            author = item.get('authors', [{}])[0].get('authorName', '')
            publisher = item.get('publisher', '')
            lowPrice = item.get('lowPrice', '')
            highPrice = item.get('highPrice', '')
            isbn = item.get('iSBN', '')
            qualityDescription = item.get('qualityDescription', '').replace(',', ' ')

            file.write(f'{name},{author},{publisher},{lowPrice},{highPrice},{isbn},{qualityDescription}\n')

    print(f"Page {page} done.")
    time.sleep(1)  # be polite to the server